In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import dask.dataframe as dd
import re
from dask.diagnostics import ProgressBar
import dask

Regular pandas wasn't working. Learned about dask and it worked beautifully. The below is only needed the first time. Once you save the filtered csv you can skip the steps to filter it down to just 2018 onwards.

In [ ]:
#seattle_df = dd.read_csv('..\data\checkouts_by_title.csv', dtype={'ISBN': 'object'})

In [ ]:
#filtered_seattle_df = seattle_df[seattle_df['CheckoutYear'] >= 2018]

In [ ]:
#filtered_seattle_df.head()

In [ ]:
#result = filtered_seattle_df.compute()

In [ ]:
#result.to_csv('..\\data\\filtered_checkouts_by_title.csv', index=False)

Steps to load and run from the filtered data that I saved to a CSV file.

In [ ]:
seattle_df = dd.read_csv('..\data\\filtered_checkouts_by_title.csv', dtype={'ISBN': 'object'})

In [ ]:
seattle_df.head()

In [ ]:
seattle_df.info()

In [ ]:
result = seattle_df.compute()

In [ ]:
result.info()

Trying to clean ISBN and split into ISBN-10 and ISBN-13. Changing to one by one so Dask doesn't throw an error. Dask did not work. ):

Finding the corresponding lengths to sort into the correct columns.

Trying for loop and chunk iterator.

In [ ]:
def process_chunk(chunk):
    chunk_copy = chunk.copy()
    chunk_copy['ISBN'] = chunk_copy['ISBN'].astype(str)
    split_isbn = chunk_copy['ISBN'].str.split(',', expand=True)
    chunk_copy.loc[:, 'Possible ISBN-10'] = split_isbn[0].str.strip()
    chunk_copy.loc[:, 'Possible ISBN-13'] = None
    if split_isbn.shape[1] > 1:
        chunk_copy.loc[:, 'Possible ISBN-13'] = split_isbn[1].str.strip()
    chunk_copy.loc[:, 'ISBN-10'] = chunk_copy['Possible ISBN-10'].where(chunk_copy['Possible ISBN-10'].str.len() == 10, np.nan)
    chunk_copy.loc[:, 'ISBN-13'] = chunk_copy['Possible ISBN-13'].where(chunk_copy['Possible ISBN-13'].str.len() == 13, chunk_copy['Possible ISBN-10'])
    chunk_copy = chunk_copy.drop(columns=['Possible ISBN-10', 'Possible ISBN-13'])
    return chunk_copy

In [ ]:
chunk_size = 10**5
chunks = [process_chunk(result[i:i+chunk_size]) for i in range(0, result.shape[0], chunk_size)]

In [ ]:
result_processed = pd.concat(chunks, ignore_index=True)

In [ ]:
result_processed.info()

In [ ]:
result_processed.head()

Checking to see if either of the ISBN columns have complete isbn data so I can possibly drop the other isbn columns.

In [ ]:
result_processed['ISBN-13'].isna().sum()

In [ ]:
result_processed['ISBN-10'].isna().sum()

In [ ]:
result_processed['PublicationYear'].isna().sum()

ISBN-13 has a record for every book so I am dropping the ISBN and the ISBN-10 columns. **Not doing this anymore because this was a LIE lol.**

In [ ]:
result_processed = result_processed.drop(columns=['ISBN', 'ISBN-10'])

In [ ]:
result_processed.head(5)

Trying to clean the year column. Decided to keep null values so I can possibly do something with them if I want to.

In [ ]:
def clean_year(year):
    if pd.isna(year):
        return year 
    year = str(year) 
    cleaned_year = ''.join(filter(str.isdigit, year))
    return cleaned_year[:4] if len(cleaned_year) >= 4 else None

In [ ]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [ ]:
result_processed['Cleaned_Year'].unique()

Discovered that there's some more values that are weird. Trying something else.

In [ ]:
def clean_year(year):
    if pd.isna(year):
        return np.nan
    year = str(year)
    year = ''.join(filter(lambda x: x.isdigit(), year))
    year = year[-4:]
    return year if len(year) == 4 else np.nan

In [ ]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [ ]:
result_processed['Cleaned_Year'].unique()

Trying to see what is going on with the weird rows.

In [ ]:
result_processed[result_processed['Cleaned_Year'] == '92009']

Adding more cases to clean this further.

In [ ]:
def clean_year(year):
    if isinstance(year, str):
        match = re.search(r"\[(\d{4})-(\d{2,4})\]", year)
        if match:
            return match.group(1)  
        
        match = re.match(r"(\d{4})(\d{4})", year)
        if match:
            return match.group(1)
        
    return year

In [ ]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [ ]:
result_processed['Cleaned_Year'] = pd.to_numeric(result_processed['Cleaned_Year'], errors='coerce')

In [ ]:
result_processed['Cleaned_Year'].unique()

In [ ]:
result_processed[result_processed['Cleaned_Year'] == 2022.1]

Gonna replace some of the easy ones.Run as many times as you need to clean up the above lol.

In [ ]:
indexes_to_update = [12303238]

In [ ]:
new_year = 2022

In [ ]:
result_processed.loc[indexes_to_update, 'PublicationYear'] = new_year

In [ ]:
result_processed.loc[indexes_to_update, 'Cleaned_Year'] = new_year

Combining all the cleaning fuctions for next time I run this notebook.

In [ ]:
def clean_year(year):
    if pd.isna(year):
        return np.nan
    
    year = str(year)
    match = re.search(r"\[(\d{4})-(\d{2,4})\]", year)
    if match:
        return match.group(1)
    
    match = re.match(r"(\d{4})(\d{4})", year)
    if match:
        return match.group(1)
    
    year = ''.join(filter(lambda x: x.isdigit(), year))
    return year[-4:] if len(year) == 4 else np.nan

In [ ]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [ ]:
result_processed['Cleaned_Year'] = pd.to_numeric(result_processed['Cleaned_Year'], errors='coerce')

In [ ]:
result_processed['Cleaned_Year'].unique()

Changing back to integers.

In [ ]:
result_processed['Cleaned_Year'] = result_processed['Cleaned_Year'].astype('Int64')

In [ ]:
result_processed.head(20)

Saving to a CSV to not lose all my manual cleaning :)

In [ ]:
result_processed.to_csv('cleaned_checkouts_data.csv', index=False)

Starting to work on open library api. Seems like I need to fetch work ids first and then match the ratings to the work ids.

In [ ]:
def find_work_id_by_isbn(isbn):
    url = f"https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json"
    response = requests.get(url)
    data = response.json()
    isbn_key = f"ISBN:{isbn}"
    
    if isbn_key in data:
        book_info = data[isbn_key]
        
        if 'key' in book_info:
            return book_info['key']
        else:
            return None

In [ ]:
isbn = "9780061754159"  
work_id = find_work_id_by_isbn(isbn)
print(work_id)

In [ ]:
work_ids = []

In [ ]:
for index, row in result_processed.iterrows():
    isbn_10 = row['ISBN-10']
    isbn_13 = row['ISBN-13']
    
    if not pd.isna(isbn_10):
        work_id = find_work_id_by_isbn(isbn_10)
        work_ids.append(work_id)
    
    if not pd.isna(isbn_13):
        work_id = find_work_id_by_isbn(isbn_13)
        work_ids.append(work_id)

In [ ]:
work_ids

Need to revisit this I don't think I can do 12 million rows worth of API requests lol.

Loading the cleaned data.

In [2]:
dtypes = {
    'ISBN': 'object',
    'ISBN-10': 'object',
    'ISBN-13': 'object'
}
seattle_df = dd.read_csv('..\data\cleaned_checkouts_data.csv', dtype=dtypes)

In [3]:
result = seattle_df.compute()

In [4]:
result.head(20)

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,ISBN,Creator,Subjects,Publisher,PublicationYear,ISBN-10,ISBN-13,Cleaned_Year
0,Digital,OverDrive,EBOOK,2023,3,2,Harry Potter und der Stein der Weisen,9781781100769,J. K. Rowling,"Juvenile Fiction, Juvenile Literature",Pottermore,2020,NaN,9781781100769,2020.0
1,Digital,OverDrive,AUDIOBOOK,2022,9,2,"The Free Bastards: The Lot Lands Series, Book ...",9780593414682,Jonathan French,"Fantasy, Fiction",Books on Tape,2021,NaN,9780593414682,2021.0
2,Digital,OverDrive,EBOOK,2022,9,8,"Dawn: Warriors: The New Prophecy Series, Book 3",9780061757433,Erin Hunter,"Fantasy, Juvenile Fiction, Juvenile Literature",HarperCollins Publishers Inc.,2007,NaN,9780061757433,2007.0
3,Physical,Horizon,BOOK,2022,9,1,Black box : a record of the catastrophe. Volum...,"162963123X, 9781629631233",NaN,"Social change, Social change Literary collections","PM Press,",[2015],162963123X,9781629631233,2015.0
4,Digital,OverDrive,AUDIOBOOK,2023,3,2,Blood Games (unabridged),9781452698052,Chloe Neill,"Fantasy, Fiction, Literature, Romance","Tantor Media, Inc",2015,NaN,9781452698052,2015.0
5,Physical,Horizon,BOOK,2022,9,1,Instant Pot miracle : from gourmet to everyday...,"1328851052, 9781328851055",NaN,"Pressure cooking, Smart cookers, Cookbooks","Houghton Mifflin Harcourt,",[2017],1328851052,9781328851055,2017.0
6,Physical,Horizon,BOOK,2022,9,3,The 26-story treehouse / Andy Griffiths ; illu...,"1250026911, 9781250026910","Griffiths, Andy, 1961-","Tree houses Juvenile fiction, Storytelling Juv...","Feiwel and Friends,",2014.,1250026911,9781250026910,2014.0
7,Digital,OverDrive,EBOOK,2023,3,1,"Strong, Silent Cowboy",9781250088406,Lora Leigh,"Fiction, Romance, Western",Macmillan Publishers,2021,NaN,9781250088406,2021.0
8,Digital,OverDrive,AUDIOBOOK,2023,3,9,Being Peace (unabridged),9781615440078,Thich Nhat Hanh,"Religion & Spirituality, Nonfiction",Better Listen LLC,2009,NaN,9781615440078,2009.0
9,Digital,OverDrive,EBOOK,2022,9,2,The Best American Science and Nature Writing 2015,9780544286757,Rebecca Skloot,"Literary Criticism, Nature, Science, Nonfiction",Open Road Integrated Media,2015,NaN,9780544286757,2015.0


Trying to work on subjects cleaning.

In [5]:
result['Subjects'].nunique()

432458

This is to be able to see more when I run the head or tail functions.

In [6]:
pd.set_option('display.max_rows', None)

This is to put the limit back so I don't accidentally blow up my pc.

In [7]:
#pd.reset_option('display.max_rows')

In [8]:
result['Subjects'].value_counts().head(200)

Fiction, Literature                                                                291799
Fiction, Mystery                                                                   253494
Juvenile Fiction, Juvenile Literature                                              181872
Fiction, Romance                                                                   115703
Fiction, Mystery, Suspense, Thriller                                               100156
Cooking & Food, Nonfiction                                                          87651
Fantasy, Fiction                                                                    78182
Fiction, Science Fiction                                                            75585
Fiction, Literature, Romance                                                        73249
Fiction, Historical Fiction, Romance                                                70905
Fiction, Thriller                                                                   68531
Fiction, S

In [9]:
result['Subjects'].value_counts().tail(50)

Sociology Urban, Cities and towns United States                                                                                                                                                                                                                                                                                           1
United States Secret Service Juvenile literature, Presidents Protection United States Juvenile literature                                                                                                                                                                                                                                 1
English literature Early modern 1500 1700 History and criticism, English literature Women authors History and criticism, Women and literature England History 16th century, Women and literature England History 17th century, Women England History Renaissance 1450 1600, Women in literature, Renaissance England, Canon Literature    1
Isla

Trying to categorize based on keywords.

Add more if this doesn't sort things enough.

In [22]:
def clean_subject(subject):
    if isinstance(subject, str):
        keywords = [word.strip().lower() for word in subject.split(',')]
        
        primary_category = 'Fiction' if 'fiction' in keywords or 'drama' in keywords else 'Nonfiction'
        
        category_mapping = {
            'operas': 'Music',
            'history': 'History',
            'romance': 'Romance',
            'juvenile': 'Juvenile Literature',
            'cooking': 'Cooking & Food'
        }
        
        secondary_category = None
        
        for word in keywords:
            for key, value in category_mapping.items():
                if key in word:
                    secondary_category = value
                    break  
            
            if secondary_category: 
                break
        
        return primary_category, (secondary_category or 'General')
    
    return 'Uncategorized', 'Uncategorized'

In [23]:
ddf = dd.from_pandas(result, npartitions=8) 

In [24]:
ddf['Cleaned_Categories'] = ddf['Subjects'].apply(clean_subject, meta=('x', 'f8'))

This was just to diagnose what was going on with the pairs.

In [13]:
#print(ddf['Cleaned_Categories'].compute().unique())

[('Nonfiction', 'Juvenile Literature') ('Nonfiction', 'Music')
 ('Nonfiction', 'General') ('Fiction', 'General') ('Fiction', 'Romance')
 ('Nonfiction', 'History') ('Uncategorized', 'Uncategorized')
 ('Nonfiction', 'Romance') ('Fiction', 'Juvenile Literature')
 ('Fiction', 'History')]


In [25]:
def extract_category(categories, position):
    if categories and isinstance(categories, tuple) and len(categories) > position:
        return categories[position]
    else:
        return None

In [26]:
ddf['Cleaned_Primary'] = ddf['Cleaned_Categories'].apply(lambda x: extract_category(x, 0), meta=('x', 'O'))
ddf['Cleaned_Secondary'] = ddf['Cleaned_Categories'].apply(lambda x: extract_category(x, 1), meta=('x', 'O'))

In [27]:
ddf = ddf.drop(columns=['Cleaned_Categories'])

In [28]:
with ProgressBar():
    result_dask = ddf.compute()

[########################################] | 100% Completed | 89.94 s


In [29]:
result_dask.head(50)

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,ISBN,Creator,Subjects,Publisher,PublicationYear,ISBN-10,ISBN-13,Cleaned_Year,Cleaned_Primary,Cleaned_Secondary
0,Digital,OverDrive,EBOOK,2023,3,2,Harry Potter und der Stein der Weisen,9781781100769,J. K. Rowling,"Juvenile Fiction, Juvenile Literature",Pottermore,2020,NaN,9781781100769,2020.0,Nonfiction,Juvenile Literature
0,Physical,Horizon,SOUNDDISC,2018,7,1,Ca ira = There is hope / Roger Waters.,NaN,"Waters, Roger","Operas, France History Revolution 1789 1799 Mu...","Sony Classical/Columbia,",[2005],NaN,NaN,2005.0,Nonfiction,Music
0,Digital,OverDrive,EBOOK,2019,1,3,Jacques Pépin Heart & Soul in the Kitchen,NaN,Jacques Pépin,"Cooking & Food, Nonfiction",Houghton Mifflin Harcourt Trade and Reference,2015,NaN,NaN,2015.0,Nonfiction,Cooking & Food
0,Physical,Horizon,SOUNDDISC,2019,9,1,Unrepentant Geraldines / Tori Amos.,NaN,"Amos, Tori",Popular music 2011 2020,"Mercury Classics,",[2014],NaN,NaN,2014.0,Nonfiction,General
0,Physical,Horizon,BOOK,2022,11,4,Car record breakers : Fastest! Biggest! Most e...,"178312380X, 9781783123803","Virr, Paul,","Automobiles Statistics Juvenile literature, Ex...","Carlton Kids,",2018.,178312380X,9781783123803,2018.0,Nonfiction,Juvenile Literature
0,Physical,Horizon,VIDEODISC,2020,12,1,The way back / a Newmarket Films in associatio...,NaN,NaN,World War 1939 1945 Prisoners and prisons Dram...,"Image Entertainment,",[2011],NaN,NaN,2011.0,Nonfiction,General
0,Digital,OverDrive,EBOOK,2018,11,1,Desmond Pucket and the Mountain Full of Monste...,NaN,Mark Tatulli,"Comic and Graphic Books, Humor (Fiction), Juve...","Andrews McMeel Publishing, LLC",2014,NaN,NaN,2014.0,Nonfiction,Juvenile Literature
0,Physical,Horizon,BOOK,2019,8,3,More all-of-a-kind family / Sydney Taylor ; il...,NaN,"Taylor, Sydney, 1904-1978,","Jews New York State New York Juvenile fiction,...","Lizzie Skurnick Books,",[2014],NaN,NaN,2014.0,Nonfiction,Juvenile Literature
0,Digital,OverDrive,AUDIOBOOK,2019,7,3,The Dakota Winters: A Novel (unabridged) (Unab...,NaN,Tom Barbash,"Fiction, Literature",HarperCollins Publishers Inc.,2018,NaN,NaN,2018.0,Fiction,General
0,Physical,Horizon,BOOK,2019,2,2,The shadows in the street / Susan Hill.,NaN,"Hill, Susan, 1942-","Serrailler Simon Fictitious character Fiction,...","Overlook Press,",[2010],NaN,NaN,2010.0,Nonfiction,General
